In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
from matplotlib.collections import LineCollection
from sklearn.metrics import RocCurveDisplay
from pycaret.anomaly import *
from sklearn.preprocessing import MinMaxScaler

In [ ]:
col = ['eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'ip.len', 'ip.ttl', 'ip.proto', 'attack']


In [ ]:
def calc_entropy(value):
    uvalue,counts = np.unique(value, return_counts=True)
    if value.shape[0] > 1:
        return entropy(counts)
    else :
        return -1

def get_entropy(column,i,df):
    temp = df.loc[df['time'] == i]
    return calc_entropy(temp[column])

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
# for i in range(time):
#     result = [get_entropy(col[j],i,data) for j in range(1,len(col))]
#     if -1 not in result:
#         file_entropy[i,1:] = result

# file_entropy = file_entropy[~np.any(file_entropy == -1, axis=1)]
# file_entropy = file_entropy[~np.all(file_entropy == 0, axis=1)]
# attack_entropy = file_entropy[:,-1]
    
# to_file = pd.DataFrame(file_entropy)
# to_file.columns = col
# to_file.to_csv('training_entropy.csv', index=False)
# attack_to_file = pd.DataFrame(attack_entropy)
# attack_to_file.columns = ['attack']
# attack_to_file.to_csv('training_attack.csv', index=False)


file_entropy = pd.read_csv('testsets/entropy/training_entropy.csv')[['eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'ip.len', 'ip.ttl', 'ip.proto', 'attack']].to_numpy()
attack_entropy = pd.read_csv('testsets/entropy/training_attack.csv').to_numpy()

In [ ]:
   attack_time = np.asarray(np.where(attack_entropy > 0))[0]      

In [ ]:
smooth_entropy = file_entropy.copy()
for i in range(smooth_entropy.shape[1]):
    smooth_entropy[:,i] = smooth(smooth_entropy[:,i],1000)

smooth_entropy = smooth_entropy[1000:-500,:]
for j in range(len(smooth_entropy[:,-1])):
    if smooth_entropy[j,-1] > 0:
        smooth_entropy[j,-1] = 1

In [ ]:
smooth_time = np.asarray(np.where(smooth_entropy[:,-1] == 1))[0]

In [ ]:
# for i in range(file_entropy.shape[1]-1):
#     x = range(file_entropy.shape[0])
#     y = file_entropy[:,i]
#     points = np.array([x, y]).T.reshape(-1, 1, 2)
#     segments = np.concatenate([points[:-1], points[1:]], axis=1)

#     cm = dict(zip(range(0,2,1),list("br")))
#     colors = list( map( cm.get , np.isin(x,attack_time) ))

#     lc = LineCollection(segments, colors=colors, linewidths= 1)
#     fig, ax = plt.subplots()
#     ax.add_collection(lc)

#     ax.autoscale()
#     ax.margins(0.1)
#     plt.xlabel('Time (second)')
#     plt.ylabel('Entropy')
#     plt.title(col[i])
#     plt.savefig('plot/all/entropy/all_'+col[i]+'_entropy.pdf') 
#     plt.show() 

In [ ]:
for i in range(0,len(col)-1):
# for i in range(0,1):
    x = range(smooth_entropy.shape[0])
    y = smooth_entropy[:,i]
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    cm = dict(zip(range(0,2,1),list("br")))
    colors = list( map( cm.get , np.isin(x,smooth_time) ))

    lc = LineCollection(segments, colors=colors, linewidths=1)
    fig, ax = plt.subplots()
    ax.add_collection(lc)
    ax.autoscale()
    ax.margins(0.1)
    plt.xlabel('Time (second)')
    plt.ylabel('Entropy')
    plt.title(col[i])
#     plt.savefig('plot/all/smooth/all_'+col[i]+'_smoooth_entropy.pdf') 
    plt.show()

In [ ]:
smooth_entropy = pd.DataFrame(smooth_entropy)
smooth_entropy.columns = col

file_entropy = pd.DataFrame(file_entropy)
file_entropy.columns = col

In [ ]:
# plotcol = np.asarray([['eth.src', 'eth.dst', 'ip.src', 'ip.dst']
#           ,['ip.len', 'ip.id', 'ip.ttl', 'ip.proto']
#           ,['tcp.srcport', 'tcp.dstport', 'tcp.seq', 'tcp.ack']
#           ,['tcp.flags', 'tcp.window_size', 'tcp.time_delta', 'tcp.time_relative']])

# fig, axs = plt.subplots(4, 4,constrained_layout = True,figsize=(20,10))



# for i in range(len(plotcol)):
#     for j in range(len(plotcol[i,:])):
#         x = range(file_entropy.shape[0])
#         y = file_entropy[plotcol[i,j]]
#         points = np.array([x, y]).T.reshape(-1, 1, 2)
#         segments = np.concatenate([points[:-1], points[1:]], axis=1)
#         cm = dict(zip(range(0,2,1),list("br")))
#         colors = list( map( cm.get , np.isin(x,smooth_time) ))

#         lc = LineCollection(segments, colors=colors, linewidths=1)
#         axs[i, j].plot(x, y)
#         axs[i, j].set_title(plotcol[i,j])
#         axs[i, j].add_collection(lc)
#         axs[i, j].autoscale()
#         axs[i, j].margins(0.1)
#         axs[i, j].set(xlabel='Time (second)', ylabel='Entropy')
        
# plt.savefig('plot/all/entropy/all_entropy.pdf')  

In [ ]:
# plotcol = np.asarray([['eth.src', 'eth.dst', 'ip.src', 'ip.dst']
#           ,['ip.len', 'ip.id', 'ip.ttl', 'ip.proto']
#           ,['tcp.srcport', 'tcp.dstport', 'tcp.seq', 'tcp.ack']
#           ,['tcp.flags', 'tcp.window_size', 'tcp.time_delta', 'tcp.time_relative']])

# fig, axs = plt.subplots(4, 4,constrained_layout = True,figsize=(20,10))



# for i in range(len(plotcol)):
#     for j in range(len(plotcol[i,:])):
#         x = range(smooth_entropy.shape[0])
#         y = smooth_entropy[plotcol[i,j]]
#         points = np.array([x, y]).T.reshape(-1, 1, 2)
#         segments = np.concatenate([points[:-1], points[1:]], axis=1)
#         cm = dict(zip(range(0,2,1),list("br")))
#         colors = list( map( cm.get , np.isin(x,smooth_time) ))

#         lc = LineCollection(segments, colors=colors, linewidths=1)
#         axs[i, j].plot(x, y)
#         axs[i, j].set_title(plotcol[i,j])
#         axs[i, j].add_collection(lc)
#         axs[i, j].autoscale()
#         axs[i, j].margins(0.1)
#         axs[i, j].set(xlabel='Time (second)', ylabel='Entropy')
        
# plt.savefig('plot/all/smooth/all_smooth_entropy.pdf')  

In [ ]:
X = smooth_entropy.drop(['attack'],axis=1)
setup(X,session_id=123)

In [ ]:
models()

In [ ]:
mcd = create_model('mcd', fraction = 0.054)
save_model(pca,'models/mcd')

In [ ]:
cluster = create_model('cluster', fraction = 0.054)
save_model(cluster,'models/cluster')

In [ ]:
iforest = create_model('iforest', fraction = 0.054)
save_model(iforest,'models/iforest')
iforest_result = assign_model(iforest)
print(iforest_result)

In [ ]:
histogram = create_model('histogram', fraction = 0.054)
save_model(histogram,'models/histogram')

In [ ]:
svm = create_model('svm', fraction = 0.054)
save_model(svm,'models/svm')

In [ ]:
pca = create_model('pca', fraction = 0.054)
save_model(pca,'models/pca')